In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

In [ ]:
data=pd.read_csv("D:\sentiment_analysis_project\dataset\sample_dataset.csv")

In [7]:
data.shape

(50000, 2)

In [8]:
data.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [9]:
data.tail(3)

,review,sentiment
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [10]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [12]:
data.isnull().any()

review       False
sentiment    False
dtype: bool

In [13]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

# Text normalization

## tockenization

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

import spacy
import re
import string
import unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob, Word
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
#tokenization of text 
tokenizers=ToktokTokenizer()
#setting English stopwords
stopwords=nltk.corpus.stopwords.words('english')

In [21]:
from bs4 import BeautifulSoup
import re

# Removing noisy text
def noiseremoval_text(text):
    # Parse the text using BeautifulSoup to remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    
    # Remove text within square brackets (e.g., [example])
    text = re.sub(r'\[[^]]*\]', '', text)
    
    # Additional cleaning (optional, based on your use case)
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text


In [22]:
# Apply the `noiseremoval_text` function to the 'review' column
data['review'] = data['review'].apply(noiseremoval_text)


/tmp/ipykernel_30/46840122.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [23]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Stemming 

In [28]:
# Define stemming function
def stemmer(text):
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

    
# Apply the stemming function to the 'review' column
data['review'] = data['review'].apply(stemmer)

In [29]:
data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,A wonder littl production. the film techniqu i...,positive
2,I thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive


In [38]:
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer

# Define function to remove stopwords
def removing_stopwords(text, is_lower_case=False):
    # Initialize the tokenizer
    tokenizer = ToktokTokenizer()
    
    # Load English stopwords
    stop_wr = set(stopwords.words('english'))
    
    # Tokenize the text
    tokens = tokenizer.tokenize(text)
    tokens = [i.strip() for i in tokens]
    
    # Filter tokens based on stopwords
    if is_lower_case:
        filtered_tokens = [i for i in tokens if i not in stop_wr]
    else:
        filtered_tokens = [i for i in tokens if i.lower() not in stop_wr]
    
    # Join the filtered tokens back into a single string
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


In [39]:
#Apply function on review column
# Apply the stemming function to the 'review' column
data['review'] = data['review'].apply(removing_stopwords)

In [40]:
data.head()

,review,sentiment
0,one review ha mention watch 1 Oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive


# Train test split

In [42]:
#split the dataset
#train dataset
train_reviews_data=data.review[:30000]

In [44]:
train_reviews_data.shape

(30000,)

In [43]:
#test dataset
test_reviews_data=data.review[30000:]

In [46]:
test_reviews_data.shape

(20000,)

# Bag of words

In [47]:
# Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False, ngram_range=(1,3))
#transformed train reviews
cv_train=cv.fit_transform(train_reviews_data)
# transform test reviews
cv_test=cv.transform(test_reviews_data)

print('BOW_cv_train:',cv_train.shape)
print('BOW_cv_test:',cv_test.shape)
#vocab=get_feature_name()-toget feature names

BOW_cv_train: (30000, 4954557)
BOW_cv_test: (20000, 4954557)


# TF_IDF

In [48]:
# Count vectorizer for bag of words
tf=TfidfVectorizer(min_df=0,max_df=1,use_idf=True, ngram_range=(1,3))
#transformed train reviews
tf_train=cv.fit_transform(train_reviews_data)
# transform test reviews
tf_test=cv.transform(test_reviews_data)

print('Tfidf_train:',tf_train.shape)
print('Tfidf_test:',tf_test.shape)
#vocab=get_feature_name()-toget fea

Tfidf_train: (30000, 4954557)
Tfidf_test: (20000, 4954557)


# Label encoding 

In [51]:
#labeling the sentient data
label=LabelBinarizer()
#tronsformed sentiment data
sentiment_data=label.fit_transform(data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [52]:
sentiment_data

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [60]:
train_data=data.sentiment[:30000]

In [61]:
test_data=data.sentiment[30000:]

In [65]:
# Define the Logistic Regression model
logistic = LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=42)

# Fit the model using Bag of Words (BoW) features
lr_bow = logistic.fit(cv_train, train_data)  # Ensure `train_labels` is correctly defined
print(lr_bow)

LogisticRegression(C=1, max_iter=500, random_state=42)


In [67]:
#predicating the model for bag of words
lr_bow_predict=logistic.predict(cv_test)
print(lr_bow_predict)

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [75]:
#Accuracy score for bag of words
bow_score=accuracy_score(test_data,bow_)
print("bow_score:",lr_bow_score)

NameError: name 'bow_predict' is not defined

In [68]:
#Fitting the model for tfidf features
lr_tfidf=logistic.fit(tf_train,train_data)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)


In [71]:
##predicting the model for tfidf features
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [73]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(test_data,lr_bow_predict)
print("lr_bow_score:",lr_bow_score)

lr_bow_score: 0.59215


In [77]:
##Predicting the model for tfidf feature
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [81]:
#Accuracy score for tfdf features
lr_tfidf_score=accuracy_score(test_data,lr_tfidf_predict)
print("lr_tfidf_score",lr_tfidf_score)

lr_tfidf_score 0.59215
